In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
method = "molgrid"

n_frags = 25

In [3]:
import open3d as o3d
from rdkit import Chem
import numpy as np

from collections import defaultdict

import tqdm
import os, sys, re

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
sys.path.append("../../")
sys.path.append("../")

In [5]:
from utils import show, align

## Load BRD4 Inhibitors

In [6]:
path = "../ligands/BRD4"

In [7]:
files = []
for f in os.listdir(path):
    fname, ext = os.path.splitext(f)
    if (
        ext == ".pcd"
        and method in fname
        and not "tran" in fname
        and not "murcko" in fname
    ):
        files.append(os.path.join(path, f))

# Sort BRD4 ligand files by number
r = re.compile("\d{1,2}")
files.sort(key=lambda f: int(r.search(os.path.basename(f)).group()))

print(files)

pcds = []
mols = []
for f in files:
    pcd = o3d.io.read_point_cloud(f)
    pcds.append(pcd)

    s = Chem.SDMolSupplier(f.replace(".pcd", ".sdf").replace(f"_{method}", ""))
    mol = next(s)
    mols.append(mol)

['../ligands/BRD4/ligand-1_molgrid.pcd', '../ligands/BRD4/ligand-2_molgrid.pcd', '../ligands/BRD4/ligand-3_molgrid.pcd', '../ligands/BRD4/ligand-4_molgrid.pcd', '../ligands/BRD4/ligand-5_molgrid.pcd', '../ligands/BRD4/ligand-6_molgrid.pcd', '../ligands/BRD4/ligand-7_molgrid.pcd', '../ligands/BRD4/ligand-8_molgrid.pcd', '../ligands/BRD4/ligand-9_molgrid.pcd', '../ligands/BRD4/ligand-10_molgrid.pcd']


## Load Fragments

Load fragments. Only fragments that are considered to be synthetically accessible are considered here, in order to reduce computational resources:

In [8]:
fragpath = path = "../../fragments/VEHICLe-good/"
fragfiles = [
    os.path.join(fragpath, f)
    for f in os.listdir(fragpath)
    if os.path.splitext(f)[-1] == ".pcd"
]

fragfiles = fragfiles[:n_frags]

fragpcds = []
fragmols = []
for ff in tqdm.tqdm(fragfiles):
    pcd = o3d.io.read_point_cloud(ff)
    fragpcds.append(pcd)

    s = Chem.SDMolSupplier(ff.replace(".pcd", ".sdf"))
    mol = next(s)
    fragmols.append(mol)

100%|██████████| 25/25 [00:00<00:00, 1043.06it/s]


## Align Fragments to CDK2 Inhibitors

In [9]:
alignments = {}

for i, f in enumerate(files):
    name = os.path.splitext(os.path.basename(f))[0].replace(f"_{method}", "")

    alignments[name] = defaultdict(list)

    for j, ff in enumerate(tqdm.tqdm(fragfiles, desc="Fragments " + name)):
        fidx = os.path.splitext(os.path.basename(ff))[0].replace(f"fragment_", "")

        gfit, cfit, hfit = align(fragmols[j], fragpcds[j], pcds[i], hfit=True)

        alignments[name]["fragment"].append(int(fidx))
        alignments[name]["gfit"].append(gfit.fitness)
        alignments[name]["cfit"].append(cfit.fitness)
        alignments[name]["hfit"].append(hfit)
        alignments[name]["mol"].append(mols[i])
        alignments[name]["fragmol"].append(fragmols[j])

Fragments ligand-10: 100%|██████████| 25/25 [00:07<00:00,  3.49it/s]


In [10]:
import pandas as pd

d = {}
for outerKey, innerDict in alignments.items():
    for innerKey, values in innerDict.items():
        d[(outerKey, innerKey)] = values

df = pd.DataFrame.from_dict(d)
df = df.stack(level=0).swaplevel().sort_index()
df.index.names = ["lig", "idx"]

In [11]:
df["cfit + hfit"] = df["cfit"] + df["hfit"]
df

cfit  fragment  \
lig      idx                       
ligand-1 0    0.867435      1046   
         1    0.898396         0   
         2    0.779141         1   
         3    0.812500      1047   
         4    0.862687        10   
...                ...       ...   
ligand-9 20   0.720195      1006   
         21   0.818182      1055   
         22   0.827160      1007   
         23   0.778409      1056   
         24   0.941176      1008   

                                                       fragmol      gfit  \
lig      idx                                                               
ligand-1 0    <rdkit.Chem.rdchem.Mol object at 0x7fa871e6ffa0>  0.841499   
         1    <rdkit.Chem.rdchem.Mol object at 0x7fa871dbd040>  0.401070   
         2    <rdkit.Chem.rdchem.Mol object at 0x7fa871dbd0a0>  0.699387   
         3    <rdkit.Chem.rdchem.Mol object at 0x7fa871dbd100>  0.771739   
         4    <rdkit.Chem.rdchem.Mol object at 0x7fa871dbd160>  0.785075   
...                                                        ...       ...   
ligand-9 20   <rdkit.Chem.rdchem.Mol object at 0x7fa871dbd760>  0.625304   
         21   <rdkit.Chem.rdchem.Mol object at 0x7fa871dbd7c0>  0.727273   
         22   <rdkit.Chem.rdchem.Mol object at 0x7fa871dbd820>  0.728395   
         23   <rdkit.Chem.rdchem.Mol object at 0x7fa871dbd880>  0.767045   
         24   <rdkit.Chem.rdchem.Mol object at 0x7fa871dbd8e0>  0.823529   

                  hfit                                               mol  \
lig      idx                                                               
ligand-1 0    0.645533  <rdkit.Chem.rdchem.Mol object at 0x7fa871e827c0>   
         1    0.807487  <rdkit.Chem.rdchem.Mol object at 0x7fa871e827c0>   
         2    0.650307  <rdkit.Chem.rdchem.Mol object at 0x7fa871e827c0>   
         3    0.581522  <rdkit.Chem.rdchem.Mol object at 0x7fa871e827c0>   
         4    0.531343  <rdkit.Chem.rdchem.Mol object at 0x7fa871e827c0>   
...                ...                                               ...   
ligand-9 20   0.420925  <rdkit.Chem.rdchem.Mol object at 0x7fa871e82880>   
         21   0.433333  <rdkit.Chem.rdchem.Mol object at 0x7fa871e82880>   
         22   0.617284  <rdkit.Chem.rdchem.Mol object at 0x7fa871e82880>   
         23   0.551136  <rdkit.Chem.rdchem.Mol object at 0x7fa871e82880>   
         24   0.573529  <rdkit.Chem.rdchem.Mol object at 0x7fa871e82880>   

              cfit + hfit  
lig      idx               
ligand-1 0       1.512968  
         1       1.705882  
         2       1.429448  
         3       1.394022  
         4       1.394030  
...                   ...  
ligand-9 20      1.141119  
         21      1.251515  
         22      1.444444  
         23      1.329545  
         24      1.514706  

[250 rows x 7 columns]

In [12]:
def getbest(df, lig, criteria="cfit", k=3):
    return df.query("lig == @lig").sort_values(criteria, ascending=False).head(k)


def showbest(df, lig, criteria="cfit", k=10):
    best = getbest(df, lig, criteria, k)

    for idx, row in best.iterrows():
        p = show(row.fragmol, row.mol)
        print(row[criteria])
        display(p)

In [13]:
showbest(df, "ligand-7", criteria="cfit + hfit")

1.5738636363636362


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

1.555263157894737


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

1.5236768802228413


...

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

1.4697406340057637


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

1.4598765432098766


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol